In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

2022-10-24 22:14:13.833044: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-24 22:14:13.953474: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-24 22:14:13.953498: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-24 22:14:13.975394: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-24 22:14:14.694100: W tensorflow/stream_executor/platform/de

In [3]:
data = pd.read_feather('./tavbase/gs.feather')

In [16]:
def deep_learning_dnn(df_dl, dep_var, classes):
    # Separa a variável dependente das demais
    deep_feat = df_dl.drop(columns=[dep_var], axis=1)
    deep_label = df_dl[dep_var]
    # Verifica os tipos das variáveis
	    # Verifica as colunas para normalização - as demais serão discretizadas - Função Bucketize do Tensor Flow
    categorical_columns = [col for col in deep_feat.columns if len(deep_feat[col].unique()) == 2 or deep_feat[col].dtype == 'O']
    continuous_columns = [col for col in deep_feat.columns if len(deep_feat[col].unique()) > 2 and (deep_feat[col].dtype == 'int64' or deep_feat[col].dtype == 'float64')]    
    cols_to_scale = continuous_columns[:]
    #cols_to_scale.remove('meses')
    # Ajusta as bases de treino e de teste
    XX_T = df_dl.drop(columns=[dep_var], axis=1)
    XX_t = df_dl.drop(columns=[dep_var], axis=1)
    yy_T = df_dl[dep_var]
    yy_t = df_dl[dep_var]
    # Normaliza as variáveis nas bases de treino e teste
    scaler = StandardScaler()
    XX_T.loc[:, cols_to_scale] = scaler.fit_transform(XX_T.loc[:, cols_to_scale])
    XX_t.loc[:, cols_to_scale] = scaler.fit_transform(XX_t.loc[:, cols_to_scale])
    # Ajustes das Variáveis Categórica - Não presentes neste modelo
    categorical_object_feat_cols = [tf.feature_column.embedding_column(
    tf.feature_column.categorical_column_with_hash_bucket(key=col, hash_bucket_size=1000), dimension=len(df_dl[col].unique()))
    for col in categorical_columns if df_dl[col].dtype == 'O']
    # Ajustes das Variáveis Categórica - Não presentes neste modelo
    categorical_integer_feat_cols = [tf.feature_column.embedding_column(
    tf.feature_column.categorical_column_with_identity(key=col, num_buckets=2), dimension=len(df_dl[col].unique()))
    for col in categorical_columns if df[col].dtype=='int64']
    continuous_feat_cols = [tf.feature_column.numeric_column(key=col) for col in continuous_columns] 
    feat_cols = categorical_object_feat_cols + \
                categorical_integer_feat_cols + \
                continuous_feat_cols
    # Rotina de DNN (Deep Neural Network)
    input_fun = tf.compat.v1.estimator.inputs.pandas_input_fn(XX_T, yy_T, batch_size=50, num_epochs=1000, shuffle=True)
    pred_input_fun = tf.compat.v1.estimator.inputs.pandas_input_fn(XX_t, batch_size=50, shuffle=False)
    DNN_model = tf.estimator.DNNClassifier(hidden_units=[10, 10, 10], feature_columns=feat_cols, n_classes=classes)
    DNN_model.train(input_fn=input_fun, steps=5000)
    # Resgata os resultados da DNN
    predictions = DNN_model.predict(pred_input_fun)
    pred = list(predictions)
    return pred


In [12]:
## Aula 14 - Probabilidade
dimensao = 'Country'
medidas = ['Sales', 'Quantity', 'Profit']
grupo = data.groupby(dimensao)[medidas].mean().reset_index()
grupo['Benefit'] = grupo['Profit'].apply(lambda x : 0 if x < 0 else 1)
grupo = grupo.set_index(dimensao)

In [17]:
probabilidade = deep_learning_dnn(grupo, 'Benefit', 2)



INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpfbxii4nj', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initia

2022-10-24 22:42:49.537836: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-24 22:42:49.537881: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-24 22:42:49.537904: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (marcobutzke-tavapp-pqizjxmqzbt): /proc/driver/nvidia/version does not exist


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.


2022-10-24 22:42:50.193859: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-24 22:42:50.202295: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-10-24 22:42:50.209975: W tensorflow/core/common_runtime/forward_type_inference.cc:332] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT64
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
  

INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpfbxii4nj/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.7355423, step = 0
INFO:tensorflow:global_step/sec: 504.432
INFO:tensorflow:loss = 0.69622976, step = 100 (0.199 sec)
INFO:tensorflow:global_step/sec: 668.953
INFO:tensorflow:loss = 0.68157715, step = 200 (0.150 sec)
INFO:tensorflow:global_step/sec: 710.74
INFO:tensorflow:loss = 0.6602306, step = 300 (0.141 sec)
INFO:tensorflow:global_step/sec: 690.158
INFO:tensorflow:loss = 0.65447235, step = 400 (0.145 sec)
INFO:tensorflow:global_step/sec: 666.468
INFO:tensorflow:loss = 0.65201867, step = 500 (0.150 sec)
INFO:tensorflow:global_step/sec: 657.322
INFO:tensorflow:loss = 0.6244853, step = 600 (0.152 sec)
INFO:tensorflow:global_step/sec: 702.194
INFO:tensorflow:loss = 0.62229437, step = 700 (0.142 sec)
INFO:tensorflow:global_step/sec: 613.866
IN

In [18]:
probabilidade_classe = []
for i in range(len(probabilidade)):
    probabilidade_classe.append(probabilidade[i]["class_ids"][0])
probabilidade_prob0 = []
for i in range(len(probabilidade)):
    probabilidade_prob0.append(probabilidade[i]["probabilities"][0])
probabilidade_prob1 = []
for i in range(len(probabilidade)):
    probabilidade_prob1.append(probabilidade[i]["probabilities"][1]) 
grupo['dl_classe'] = probabilidade_classe
grupo['lucro_0'] = probabilidade_prob0
grupo['lucro_1'] = probabilidade_prob1
grupo = grupo.reset_index()
grupo

,Country,Sales,Quantity,Profit,Benefit,dl_classe,lucro_0,lucro_1
0,Afghanistan,394.060364,4.145455,99.278182,1,1,0.363380,0.636620
1,Albania,243.007500,2.500000,44.332500,1,1,0.347405,0.652595
2,Algeria,184.140765,2.316327,46.461735,1,1,0.316185,0.683815
3,Angola,209.459016,2.598361,53.237459,1,1,0.347605,0.652395
4,Argentina,147.466111,3.856410,-47.932812,0,1,0.461637,0.538363
...,...,...,...,...,...,...,...,...
142,Venezuela,137.050668,3.989691,-57.849023,0,1,0.463706,0.536294
143,Vietnam,248.302639,3.758491,-7.057474,0,1,0.446557,0.553443
144,Yemen,82.190400,2.366667,-123.548600,0,1,0.448704,0.551296
145,Zambia,255.250000,2.460784,68.644412,1,1,0.328345,0.671655


In [ ]:
grupo.to_feather('./tavbase/probabilidade_pais.feather')
